In [1]:
import glob
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
%matplotlib inline

/home/anirudh/yes/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/anirudh/yes/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from matplotlib.pyplot import specgram

In [3]:
os.chdir("/media/anirudh/New Volume/Anirudh/SNU BLACKBOARD/OUR stuff/UrbanSound8K/audio/fold1")

In [4]:
def load_sound_files(file_paths):
    raw_sounds = []
    for fp in file_paths:
        #load the file
        X,sr = librosa.load(fp)
        raw_sounds.append(X)
    return raw_sounds

In [5]:
def plot_specgram(sound_names, raw_sounds):
    i = 1
    fig = plt.figure(figsize=(25,60), dpi=900)
    for n,f in zip(sound_names,raw_sounds):
        plt.subplot(10,1,i)
        specgram(np.array(f),Fs =22050)
        plt.title(n.title())
        i += 1
    plt.suptitle("Figure 2: Spectogram", x = 0.5, y = 0.915, fontsize = 18)
    plt.show()

In [6]:
def plot_waves(sound_names, raw_sounds):
    i = 1
    fig = plt.figure(figsize=(25,60), dpi=900)
    for n,f in zip(sound_names,raw_sounds):
        plt.subplot(10,1,i)
        librosa.display.waveplot(np.array(f), sr = 22050)
        plt.title(n.title())
        i += 1
    plt.suptitle("Figure 1: Waveplot",x = 0.5, y = 0.915, fontsize = 18)
    plt.show()
    

In [7]:
def plot_log_power_specgram(sound_names,raw_sounds):
    i = 1
    fig = plt.figure(figsize=(25,60),dpi = 900)
    for n,f in zip(sound_names, raw_sounds):
        plt.subplot(10,1,i)
        D = librosa.logamplitude(np.abs(librosa.stft(f))**2, ref_power=np.max)
        librosa.display.specshow(D,x_axis = 'time',y_axis = 'log')
        plt.title(n.title())
        i += 1
    plt.suptitle("Figure 3: Log power spectogram", x = 0.5, y = 0.915, fontsize = 18)
    plt.show()

In [8]:
sound_file_paths = ["57320-0-0-7.wav","24074-1-0-3.wav","15564-2-0-1.wav","31323-3-0-1.wav","46669-4-0-35.wav",
                   "89948-5-0-0.wav","40722-8-0-4.wav","103074-7-3-2.wav","106905-8-0-0.wav","108041-9-0-4.wav"]

sound_names = ["air conditioner","car horn","children playing","dog bark","drilling","engine idling","gun shot",
              "jackhammer","siren","street music"]

raw_sounds = load_sound_files(sound_file_paths)

Passing only a part of the raw_sounds list as it is very huge

In [12]:
sub_raw = raw_sounds[0][0:44100]
raw_sounds[0].shape
raw_sounds[0][0:44].shape

(44,)

In [ ]:
plot_waves(sound_names,sub_raw)
plot_specgram(sound_names,sub_raw)
plot_log_power_specgram(sound_names,sub_raw)

# We will now start extracting features:

The features of our interest will be:

        1) melspectogram: Compute a Mel-scaled power spectogram
       
        2) mfcc: Mel-frequency cepstral coefficients
        
        3) chorma-stf: Compute a chromagram form a waveform or power spectogram
        
        4) spectral_constrast: compute a spectral contrast 
        
        5) tonnetz: computes the tonal centroid features(tonnetz)

In [32]:
def extract_feature(file_name):
    X, sample_rate = librosa.load(file_name)
    stft = np.abs(librosa.stft(X))
    mfccs = np.mean(librosa.feature.mfcc(y = X, sr = sample_rate,n_mfcc = 40).T,axis = 0)
    chroma = np.mean(librosa.feature.melspectogram(X,sr = sample_rate).T,axis=0)
    mel = np.mean(librosa.feature.melspectogram(X, sr = sample_rate).T, axis = 0)
    contrast = np.mean(librosa.feature.spectral_contrast(S = stft, sr =sample_rate).T, axis = 0)
    tonnetz = np.mean(librosa.feature.tonnetz(y = librosa.effects.harmonic(X),sr =sample_rate).T,axis = 0)
    return mfccs,chroma,mel,contrast,tonnetz

In [33]:
def parse_audio_files(parent_dir, sub_dirs, file_ext = "*.wav"):
    features,labels = np.empty((0,193)),np.empty(0)
    for label, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir,sub_dir,file_ext)):
            try:
                mfccs, chroma,mel,contrast,tonnetz = extract_feature(fn)
            except Exception as e:
                print("Error encountered while parsing files:",fn)
                continue
            ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
            features = np.vstack([features,ext_features])
            labels = np.append(labels,fn.split('/')[2].split('-')[1])
    return np.array(features),np.array(labels,dtype = np.int)

In [40]:
features, labels = np.empty((0,193)),np.empty(0)
for label, sub_dir in enumerate(tr_sub_dirs):
    for fn in glob.glob(os.path.join(parent_dir,sub_dir,"*.wav")):
        mfccs,chroma, mel, contrast, tonnetz = extract_feature(fn)
        ext_features = np.hstack([mfccs,chroma,mel,contrast,tonnetz])
        features = np.vstack([features,ext_features])
        labels =

In [39]:
def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels),labels] = 1
    return one_hot_encode

In [41]:
parent_dir = 'audio'
tr_sub_dirs = ["fold1","fold2"]
ts_sub_dirs = ["fold3"]
tr_features, tr_labels = parse_audio_files(parent_dir,tr_sub_dirs)
ts_features, ts_labels = parse_audio_files(parent_dir, ts_sub_dirs)

tr_labels = one_hot_encode(tr_labels)
ts_labels = one_hot_encode(ts_labels)

In [43]:
tr_labels

array([], shape=(0, 0), dtype=float64)

In [ ]:
features, labels = np.empty

In [ ]:
tr_features

ls: cannot access '/home/anirudh/notebook/logs': No such file or directory
